<a href="https://colab.research.google.com/github/rednaviblue/LearningsPublic/blob/Geomatics/docs/maplibre/AlphaEarth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=maplibre/AlphaEarth.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/maplibre/AlphaEarth.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/leafmap/HEAD)

**Visualize AlphaEarth satellite embeddings in 3D**

Google DeepMind has released a new satellite embedding dataset called AlphaEarth. This dataset contains annual satellite embeddings from 2017 to 2024, with each pixel representing a 10x10 meter area. The dataset is available on Google Earth Engine, and can be used to train machine learning models to classify satellite imagery.

- News release: https://deepmind.google/discover/blog/alphaearth-foundations-helps-map-our-planet-in-unprecedented-detail/
- Dataset: https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_SATELLITE_EMBEDDING_V1_ANNUAL#description
- Paper: https://storage.googleapis.com/deepmind-media/DeepMind.com/Blog/alphaearth-foundations-helps-map-our-planet-in-unprecedented-detail/alphaearth-foundations.pdf
- Blog post: https://medium.com/google-earth/ai-powered-pixels-introducing-googles-satellite-embedding-dataset-31744c1f4650
- Tutorials: https://developers.google.com/earth-engine/tutorials/community/satellite-embedding-01-introduction
- Similarity search: https://earthengine-ai.projects.earthengine.app/view/embedding-similarity-search
- Clustering: https://code.earthengine.google.com/b0871454add885294f633f731b90f946


Uncomment the following line to install [leafmap](https://leafmap.org) if needed.

In [ ]:
%pip install -U leafmap

In [6]:
import ee
import leafmap.maplibregl as leafmap

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


To use the AlphaEarth satellite embeddings, you will need to authenticate with Earth Engine.

If you don't have an Earth Engine account, you can create one at https://earthengine.google.com.

Once you have an Earth Engine account, you can authenticate with Earth Engine by running the following code:

In [9]:
ee.Authenticate()
ee.Initialize(project="project-testred")

In [ ]:
m = leafmap.Map(projection="globe", sidebar_visible=True)
m.add_basemap("USGS.Topographic") # Changed basemap to USGS.Topographic
m.add_basemap("OpenTopoMap")
m.add_basemap("OpenStreetMap")
m.add_basemap("CartoDB.DarkMatter")
m.add_basemap("CartoDB.Positron")
m.add_alphaearth_gui()
m

![](https://github.com/user-attachments/assets/fdcf844e-6385-4e62-a49f-363c00fa0998)

In [ ]:
m = leafmap.Map(projection="globe", sidebar_visible=True)
m.add_basemap("USGS.Imagery")
m

In [20]:
lon = 125.61641740762558
lat = 7.086001503364461
m.set_center(lon, lat, zoom=12)

In [17]:
point = ee.Geometry.Point(lon, lat)
dataset = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")

In [18]:
image1 = dataset.filterDate("2017-01-01", "2018-01-01").filterBounds(point).first()
image2 = dataset.filterDate("2024-01-01", "2025-01-01").filterBounds(point).first()

In [25]:
vis_params = {"min": -0.22145328719723184, "max": 0.2069357939254133, "bands": ["A01", "A16", "A09"]}
m.add_ee_layer(image1, vis_params, name="Year 1 embeddings")
m.add_ee_layer(image2, vis_params, name="Year 2 embeddings")

In [ ]:
dot_prod = image1.multiply(image2).reduce(ee.Reducer.sum())

In [ ]:
vis_params = {"min": 0, "max": 1, "palette": ["white", "black"]}
m.add_ee_layer(dot_prod, vis_params, name="Similarity")
m

![](https://github.com/user-attachments/assets/f7613474-f097-483e-9b24-a188c9d0d430)

# Task
Determine the appropriate visualization range for a 10-hectare area based on the distribution of embedding values within that area.

## Define the area of interest

### Subtask:
Create an Earth Engine Geometry representing your 10-hectare area.


**Reasoning**:
Calculate the radius for a 10-hectare circle and create an Earth Engine Geometry representing the 10-hectare area around the existing point.



In [21]:
import math

# Calculate the radius for a 10-hectare circle (100,000 sq meters)
area_sq_meters = 100000
radius_meters = math.sqrt(area_sq_meters / math.pi)

# Create an Earth Engine Geometry representing the 10-hectare area
aoi = ee.Geometry.Point(lon, lat).buffer(radius_meters)

# Print the area of interest
display(aoi.getInfo())

{'type': 'Polygon',
 'coordinates': [[[125.61641740762559, 7.087607012740675],
   [125.61596551437479, 7.0875431132493345],
   [125.6155495921782, 7.087356501225547],
   [125.61520274868434, 7.087062031122742],
   [125.61495259269189, 7.0866831429404975],
   [125.61481903647177, 7.086249996344586],
   [125.61481271079577, 7.085797069905619],
   [125.61493411882188, 7.085360416573149],
   [125.61517359615716, 7.084974793859316],
   [125.61551208024126, 7.084670897169957],
   [125.61592262777768, 7.084472916500918],
   [125.6163725594086, 7.084396610970729],
   [125.6168260609213, 7.084448054439311],
   [125.61724703395176, 7.084623152047529],
   [125.61760196930335, 7.0849079661552],
   [125.61786261419866, 7.0852798257368645],
   [125.6180082211768, 7.08570913094004],
   [125.61802719963089, 7.086161709180347],
   [125.61791803851133, 7.086601535244968],
   [125.6176894267192, 7.08699359889486],
   [125.61735956157068, 7.087306691706076],
   [125.61695470034788, 7.087515891309445],
   

## Sample the embedding image

### Subtask:
Extract the embedding values from the `image1` within the defined area of interest.


**Reasoning**:
Extract the embedding values from the image within the defined area of interest using `sampleRegions` and store them in a list of dictionaries.



In [22]:
embedding_values_list = image1.sampleRegions(
    collection=aoi,
    scale=10,
    geometries=True
).getInfo()

## Analyze the data distribution

### Subtask:
Calculate statistics (e.g., minimum, maximum, percentiles) of the sampled embedding values to understand their distribution.


**Reasoning**:
Extract the embedding values from the list of dictionaries and calculate the required statistics.



In [23]:
import numpy as np

embedding_values = []
for feature in embedding_values_list['features']:
  for band, value in feature['properties'].items():
    if band.startswith('A'):  # Assuming embedding bands start with 'A'
      embedding_values.append(value)

embedding_values = np.array(embedding_values)

min_embedding = np.min(embedding_values)
max_embedding = np.max(embedding_values)
percentile_5 = np.percentile(embedding_values, 5)
percentile_95 = np.percentile(embedding_values, 95)

print(f"Minimum embedding value: {min_embedding}")
print(f"Maximum embedding value: {max_embedding}")
print(f"5th percentile of embedding values: {percentile_5}")
print(f"95th percentile of embedding values: {percentile_95}")

Minimum embedding value: -0.3553094963475586
Maximum embedding value: 0.37425605536332185
5th percentile of embedding values: -0.22145328719723184
95th percentile of embedding values: 0.2069357939254133


## Suggest visualization parameters

### Subtask:
Based on the data distribution, recommend a suitable `min` and `max` range for visualization.


**Reasoning**:
Determine the recommended min and max visualization range based on the calculated percentiles and print the results.



In [24]:
# Determine the recommended min and max visualization range using the 5th and 95th percentiles
recommended_min_vis = percentile_5
recommended_max_vis = percentile_95

# Print the recommended visualization parameters
print(f"Recommended minimum visualization value: {recommended_min_vis}")
print(f"Recommended maximum visualization value: {recommended_max_vis}")

Recommended minimum visualization value: -0.22145328719723184
Recommended maximum visualization value: 0.2069357939254133


## Summary:

### Data Analysis Key Findings

*   The radius for a 10-hectare circular area was calculated to be approximately 178.41 meters.
*   Embedding values from the image within the 10-hectare area were successfully extracted.
*   The minimum embedding value found was -0.3553, and the maximum was 0.3743.
*   The 5th percentile of the embedding values is -0.2215, and the 95th percentile is 0.2069.
*   Using the 5th and 95th percentiles, the recommended minimum visualization value is -0.221 and the recommended maximum visualization value is 0.207.

### Insights or Next Steps

*   Using percentiles for the visualization range helps to exclude extreme outliers, providing a more representative view of the majority of the data distribution.
*   These recommended visualization parameters can now be directly applied when displaying the embedding image for the 10-hectare area in Earth Engine or other visualization platforms.


## Define the area of interest

### Subtask:
Create an Earth Engine Geometry representing your 10-hectare area.

**Reasoning**:
Calculate the radius for a 10-hectare circle and create an Earth Engine Geometry representing the 10-hectare area around the existing point.

In [28]:
import math

# Calculate the radius for a 10-hectare circle (100,000 sq meters)
area_sq_meters = 100000
radius_meters = math.sqrt(area_sq_meters / math.pi)

# Create an Earth Engine Geometry representing the 10-hectare area
aoi = ee.Geometry.Point(lon, lat).buffer(radius_meters)

# Print the area of interest
display(aoi.getInfo())

{'type': 'Polygon',
 'coordinates': [[[125.61641740762559, 7.087607012740675],
   [125.61596551437479, 7.0875431132493345],
   [125.6155495921782, 7.087356501225547],
   [125.61520274868434, 7.087062031122742],
   [125.61495259269189, 7.0866831429404975],
   [125.61481903647177, 7.086249996344586],
   [125.61481271079577, 7.085797069905619],
   [125.61493411882188, 7.085360416573149],
   [125.61517359615716, 7.084974793859316],
   [125.61551208024126, 7.084670897169957],
   [125.61592262777768, 7.084472916500918],
   [125.6163725594086, 7.084396610970729],
   [125.6168260609213, 7.084448054439311],
   [125.61724703395176, 7.084623152047529],
   [125.61760196930335, 7.0849079661552],
   [125.61786261419866, 7.0852798257368645],
   [125.6180082211768, 7.08570913094004],
   [125.61802719963089, 7.086161709180347],
   [125.61791803851133, 7.086601535244968],
   [125.6176894267192, 7.08699359889486],
   [125.61735956157068, 7.087306691706076],
   [125.61695470034788, 7.087515891309445],
   

## Sample the embedding image

### Subtask:
Extract the embedding values from the `image1` within the defined area of interest.

**Reasoning**:
Extract the embedding values from the image within the defined area of interest using `sampleRegions` and store them in a list of dictionaries.

In [29]:
embedding_values_list = image1.sampleRegions(
    collection=aoi,
    scale=10,
    geometries=True
).getInfo()

## Analyze the data distribution

### Subtask:
Calculate statistics (e.g., minimum, maximum, percentiles) of the sampled embedding values to understand their distribution.

**Reasoning**:
Extract the embedding values from the list of dictionaries and calculate the required statistics.

In [30]:
import numpy as np

embedding_values = []
for feature in embedding_values_list['features']:
  for band, value in feature['properties'].items():
    if band.startswith('A'):  # Assuming embedding bands start with 'A'
      embedding_values.append(value)

embedding_values = np.array(embedding_values)

min_embedding = np.min(embedding_values)
max_embedding = np.max(embedding_values)
percentile_5 = np.percentile(embedding_values, 5)
percentile_95 = np.percentile(embedding_values, 95)

print(f"Minimum embedding value: {min_embedding}")
print(f"Maximum embedding value: {max_embedding}")
print(f"5th percentile of embedding values: {percentile_5}")
print(f"95th percentile of embedding values: {percentile_95}")

Minimum embedding value: -0.3553094963475586
Maximum embedding value: 0.37425605536332185
5th percentile of embedding values: -0.22145328719723184
95th percentile of embedding values: 0.2069357939254133


## Suggest visualization parameters

### Subtask:
Based on the data distribution, recommend a suitable `min` and `max` range for visualization.

**Reasoning**:
Determine the recommended min and max visualization range based on the calculated percentiles and print the results.

In [31]:
# Determine the recommended min and max visualization range using the 5th and 95th percentiles
recommended_min_vis = percentile_5
recommended_max_vis = percentile_95

# Print the recommended visualization parameters
print(f"Recommended minimum visualization value: {recommended_min_vis}")
print(f"Recommended maximum visualization value: {recommended_max_vis}")

Recommended minimum visualization value: -0.22145328719723184
Recommended maximum visualization value: 0.2069357939254133


## Summary:

### Data Analysis Key Findings

* The radius for a 10-hectare circular area was calculated to be approximately 178.41 meters.
* Embedding values from the image within the 10-hectare area were successfully extracted.
* The minimum embedding value found was {min_embedding:.4f}, and the maximum was {max_embedding:.4f}.
* The 5th percentile of the embedding values is {percentile_5:.4f}, and the 95th percentile is {percentile_95:.4f}.
* Using the 5th and 95th percentiles, the recommended minimum visualization value is {recommended_min_vis:.3f} and the recommended maximum visualization value is {recommended_max_vis:.3f}.

### Insights or Next Steps

* Using percentiles for the visualization range helps to exclude extreme outliers, providing a more representative view of the majority of the data distribution.
* These recommended visualization parameters can now be directly applied when displaying the embedding image for the 10-hectare area in Earth Engine or other visualization platforms.